In [1]:
import pandas as pd
import numpy as np
import re
import time

In [2]:
import requests
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.common.keys import Keys

import warnings
warnings.filterwarnings(action='ignore')

# 전통주 list

In [2]:
df = pd.read_csv('더술_전통주리스트.csv', index_col=0)
df.head(30)

,전통주이름
0,1000억 유산균 막걸리
1,1932 새싹땅콩+햅쌀/흑미
2,2017년 햅쌀로 빚은 첫술
3,52C
4,DOK막걸리
5,G12 골디락스
6,SSOA-포도
7,The 찾을수록
8,가야 프리미엄 막걸리
9,가야곡왕주


# 네이버지식백과 전통주 가격 크롤링

In [13]:
driver = webdriver.Chrome()

In [36]:


name_li = []
title_li = []
price_li = []

for name in df['전통주이름']:
    try:

        # 전통주지식백과 page
        elem = driver.get("https://terms.naver.com/list.naver?cid=42726&categoryId=58635")
        
        # 전통주 이름 검색, 페이지 이동
        title_li.append(name)
        driver.find_element_by_css_selector('#term_query').send_keys(name)
        driver.find_element_by_css_selector('#terms_search_form > fieldset > div > input.btn_search._search').click()
        
        
        driver.find_elements_by_css_selector('#content > div.search_result_area > ul > li > div.info_area > div.subject > strong > a')[0].click()
        
        # page html로 읽기
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')

        path = '#content'

        sell = soup.select(path)[0]
        
        # 상품명이랑 가격 뽑기
        name_a = sell.select("div.section_wrap > div.headword_title > h2")[0].text
        df['상품명'] = name_a
        
        for p in sell.select('div.att_type > div > div.wr_tmp_profile > div > table > tbody > tr'):
            if p.select('th')[0].text == '가격':
                price = p.select('td')[0].text
                price = price.strip()
                df['가격'] = price
                break
                           
        name_li.append(name_a)
        price_li.append(price)
        
#         print(name)
#         print(price)
#         print('='*100)
        time.sleep(1)
    
    # 없는 전통주는 nan값처리 및 넘어가기
    except:
        name_li.append(np.nan)
        price_li.append(np.nan)
        pass
        

In [9]:
import os

In [ ]:
df

In [41]:
# dataframe 만들어서 파일로 저장
df_price = pd.DataFrame()
df_price['전통주이름'] = title_li
df_price['상품명'] = name_li
df_price['가격']=price_li

In [42]:
df_price

,전통주이름,상품명,가격
0,1000억 유산균 막걸리,1000억 유산균막걸리,"￦3,200 (가격은 판매처 별로 상이할 수 있습니다)"
1,1932 새싹땅콩+햅쌀/흑미,1932 새싹땅콩 흑미막걸리,"500ml-￦15,000(병제품), \n ..."
2,2017년 햅쌀로 빚은 첫술,국순당 2010 첫술,"500ml-￦15,000(병제품), \n ..."
3,52C,NaN,NaN
4,DOK막걸리,걍즐겨,"￦6,000 (가격은 판매처 별로 상이할 수 있습니다)"
...,...,...,...
701,황감찰,NaN,NaN
702,황도배기 햅쌀막걸리,NaN,NaN
703,황진이주,황진이주,"￦13,000(가격은 판매처 별로 상이할 수 있습니다.)"
704,희양산 막걸리 15도,희양산막걸리 9%,"￦8,000 (가격은 판매처 별로 상이할 수 있습니다)"


In [44]:
df_price.to_csv('alcohol_price.csv')

In [2]:
df_price = pd.read_csv('alcohol_price.csv')

In [3]:
df_price[df_price['상품명'].isna()].head(50)

,Unnamed: 0,전통주이름,상품명,가격
3,3,52C,NaN,NaN
5,5,G12 골디락스,NaN,NaN
6,6,SSOA-포도,NaN,NaN
25,25,경산대추약주 추,NaN,NaN
35,35,고도리 아이스와인(아이스 375),NaN,NaN
36,36,고도리 징기스칸 브랜디,NaN,NaN
46,46,고향생막걸리,NaN,NaN
48,48,고흥유자주 봄에디션,NaN,NaN
54,54,관악산생막걸리,NaN,NaN
77,77,금풍양조,NaN,NaN


In [3]:
df_p = pd.read_csv('price_sample.csv')
df_p

,Unnamed: 0,상품명,가격
0,0,1000억 유산균막걸리,"￦3,200 (가격은 판매처 별로 상이할 수 있습니다)"
1,1,1932 새싹땅콩 흑미막걸리,"500ml-￦15,000(병제품), \n ..."
2,2,국순당 2010 첫술,"500ml-￦15,000(병제품), \n ..."
3,3,걍즐겨,"￦6,000 (가격은 판매처 별로 상이할 수 있습니다)"
4,4,The 찾을수록 시리즈,"￦9,000(360ml*6병)(가격은 판매처 별로 상이할 수 있습니다.)"
...,...,...,...
614,614,홍천강 탁주,"￦10,000 (가격은 판매처 별로 상이할 수 있습니다.)"
615,615,화주,"￦13,000(가격은 판매처 별로 상이할 수 있습니다.)"
616,616,황진이주,"￦13,000(가격은 판매처 별로 상이할 수 있습니다.)"
617,617,희양산막걸리 9%,"￦8,000 (가격은 판매처 별로 상이할 수 있습니다)"


In [4]:
df_pr = pd.read_csv('alcohol_price.csv')
df_pr

,Unnamed: 0,전통주이름,상품명,가격
0,0,1000억 유산균 막걸리,1000억 유산균막걸리,"￦3,200 (가격은 판매처 별로 상이할 수 있습니다)"
1,1,1932 새싹땅콩+햅쌀/흑미,1932 새싹땅콩 흑미막걸리,"500ml-￦15,000(병제품), \n ..."
2,2,2017년 햅쌀로 빚은 첫술,국순당 2010 첫술,"500ml-￦15,000(병제품), \n ..."
3,3,52C,NaN,NaN
4,4,DOK막걸리,걍즐겨,"￦6,000 (가격은 판매처 별로 상이할 수 있습니다)"
...,...,...,...,...
701,701,황감찰,NaN,NaN
702,702,황도배기 햅쌀막걸리,NaN,NaN
703,703,황진이주,황진이주,"￦13,000(가격은 판매처 별로 상이할 수 있습니다.)"
704,704,희양산 막걸리 15도,희양산막걸리 9%,"￦8,000 (가격은 판매처 별로 상이할 수 있습니다)"
